In [1]:
# xmlrpc_server.ipynb
import copy
from xmlrpc.server import *
from xmlrpc.client import *
import datetime
import pandas as pd
import pickle


class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(("localhost", 8008), 
                            requestHandler=RequestHandler)

stats_server = ServerProxy("http://localhost:8018")

def add_log(action_name):
    try:
        stats_server.add_log(action_name, datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    except Exception:
        print("WARWING! stats server error!")

def slice_log(file_paths, action_names=[], action_start_date=datetime.datetime(1, 1, 1, 0, 0).strftime("%Y-%m-%d %H:%M:%S"), action_stop_date=datetime.datetime(9999, 12, 31, 23, 59, 59, 999999).strftime("%Y-%m-%d %H:%M:%S")):
    add_log('slice_log')
    try:
        df = pd.read_json(stats_server.get_log_df(file_paths))
        if len(action_names) != 0:
            df = df.loc[df[0].isin(action_names)]
        df = df.loc[df[1] >= action_start_date]
        df = df.loc[df[1] <= action_stop_date]
        return df.to_json()
    except Exception:
        print("WARWING! stats server error!")
        
server.register_function(slice_log, 'slice_log')

# Тест
def ping():
    add_log('ping')
    return True
server.register_function(ping, 'ping')

# Время сервера
def now():
    add_log('now')
    return datetime.datetime.now()
server.register_function(now, 'now')

# Отображение строкового вида, типа и значений
def show_type(arg):
    add_log('show_type')
    return (str(arg), str(type(arg)), arg)
server.register_function(show_type, 'type')

# Сумма
def test_sum(a, b):
    add_log('test_sum')
    return a + b
server.register_function(test_sum, 'sum')

# Степень
def test_pow(a, b):
    add_log('test_pow')
    return a**b
server.register_function(test_pow, 'pow')

# Проверка нахождения клиента в черном списке c использованием Pandas Data Frame
def black_list_check(sname):
    add_log('black_list_check')
    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf8')
    exist = any(frame['Surname'] == sname)
    if (exist == True):
        return sname + ": "+ "bad_boy"
    else:
        return sname + ": "+ "good_boy"
    
server.register_function(black_list_check, 'black_list_check')
    
def black_list_check_with_date_and_fullname(sname, nname, pname, date):
    add_log('black_list_check_with_date_and_fullname')
    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf8')
    exist = any((frame['Surname'] + frame['Name']  + frame['Patronym']  + frame['Birth']) == sname + nname + pname + date)
    print(exist)
    if (exist):
        return sname + nname + pname + " with birthday: " + date + " : "+ "bad_boy"
    else:
        return sname + nname + pname + " with birthday: " + date + " : "+ "good_boy"

server.register_function(black_list_check_with_date_and_fullname, 'black_list_check_with_date_and_fullname')

# Бинарная передача данных
def send_back_binary(bin_data):
    add_log('send_back_binary')
    data = bin_data.data
    return Binary(data)
server.register_function(send_back_binary, 'send_back_binary')

# Инверсия цвета
# На вход изображение RGB размерности (M, N, 3) со значениями 0-255
def send_back_inversion(bin_data):
    add_log('send_back_inversion')
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]
    isColored = len(img_arr.shape) == 3
    for i in range(height):
        for j in range(width):
            if isColored:
                img_arr[i][j][0] = 255 - img_arr[i][j][0]
                img_arr[i][j][1] = 255 - img_arr[i][j][1]
                img_arr[i][j][2] = 255 - img_arr[i][j][2]
            else:
                img_arr[i][j] = 255 - img_arr[i][j]
    
    pimg = pickle.dumps(img_arr)        
    return Binary(pimg)
server.register_function(send_back_inversion, 'color_inversion')


def binary(val, border):
    if val < border:
        return 0
    else:
        return 255


# Бинаризация изображения по задаваемому порогу 1-255
def binary_img(bin_data, border):
    add_log('binary_img')
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]    
    for i in range(height):
        for j in range(width):
            try:
                img_arr[i][j][0] = binary(img_arr[i][j][0], border)
                img_arr[i][j][1] = binary(img_arr[i][j][1], border)
                img_arr[i][j][2] = binary(img_arr[i][j][2], border)
            except Exception:
                img_arr[i][j] = binary(img_arr[i][j], border)
    
    pimg = pickle.dumps(img_arr)        
    return Binary(pimg)
server.register_function(binary_img, 'binary_img')

# Поворот относительно вертикали
def reverse(bin_data):
    add_log('reverse')
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]    
    for i in range(height):
        for j in range(width // 2):
            temp = copy.deepcopy(img_arr[i][j])
            img_arr[i][j] = copy.deepcopy(img_arr[i][width - j - 1])
            img_arr[i][width - j - 1] = copy.deepcopy(temp)
    
    pimg = pickle.dumps(img_arr)        
    return Binary(pimg)
server.register_function(reverse, 'reverse')

print("Listening on port 8008...")
server.serve_forever()


Listening on port 8008...


127.0.0.1 - - [10/Oct/2023 01:10:30] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2023 01:10:34] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2023 01:10:38] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2023 01:10:42] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2023 01:10:46] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2023 01:10:51] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2023 01:10:55] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2023 01:10:59] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2023 01:11:03] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2023 01:11:07] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2023 01:11:11] "POST /RPC2 HTTP/1.1" 200 -
C:\Users\pavel\AppData\Local\Temp\ipykernel_9476\2165330344.py:27: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(stats_server.get_log_df(file_

True


127.0.0.1 - - [10/Oct/2023 01:12:05] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2023 01:12:15] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2023 01:12:21] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2023 01:12:28] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2023 01:12:33] "POST /RPC2 HTTP/1.1" 200 -


KeyboardInterrupt: 